# Xabarins Overview

# Imports 

In [18]:
import pandas as pd 
import numpy as np
from unicodedata import normalize


# Code 

## Import df 

In [6]:
df_small = pd.read_csv("../src/dades/font/certificats.csv")

C:\Users\joelx\AppData\Local\Temp\ipykernel_34784\2455367483.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_small = pd.read_csv("../src/dades/font/certificats.csv")


In [20]:
df_big_raw = pd.read_csv("../data/Certificats_d_efici_ncia_energ_tica_d_edificis_20250204.csv")

C:\Users\joelx\AppData\Local\Temp\ipykernel_34784\1237903497.py:1: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  df_big_raw = pd.read_csv("../data/Certificats_d_efici_ncia_energ_tica_d_edificis_20250204.csv")


In [8]:
df_small.shape

(213291, 71)

In [9]:
df_big.shape

(1336925, 69)

In [11]:
df_small.columns

Index(['num_cas', 'adre_a', 'numero', 'escala', 'pis', 'porta', 'codi_postal',
       'poblacio', 'comarca', 'nom_provincia', 'codi_poblacio', 'codi_comarca',
       'codi_provincia', 'referencia_cadastral', 'zona_climatica',
       'metres_cadastre', 'any_construccio', 'us_edifici',
       'qualificaci_de_consum_d', 'energia_prim_ria_no_renovable',
       'qualificacio_d_emissions', 'emissions_de_co2',
       'consum_d_energia_final', 'cost_anual_aproximat_d_energia',
       'vehicle_electric', 'solar_termica', 'solar_fotovoltaica',
       'sistema_biomassa', 'xarxa_districte', 'energia_geotermica',
       'informe_ins_tecnica_edifici', 'eina_de_certificacio',
       'valor_aillaments', 'valor_finestres', 'motiu_de_la_certificacio',
       'valor_aillaments_cte', 'valor_finestres_cte', 'utm_x', 'utm_y',
       'normativa_construcci', 'tipus_tramit', 'tipus_terciari',
       'qualificaci_emissions', 'emissions_calefacci',
       'qualificaci_emissions_1', 'emissions_refrigeraci',
     

In [10]:
df_big.columns

Index(['NUM_CAS', 'ADREÇA', 'NUMERO', 'ESCALA', 'PIS', 'PORTA', 'CODI_POSTAL',
       'POBLACIO', 'COMARCA', 'NOM_PROVINCIA', 'CODI_POBLACIO', 'CODI_COMARCA',
       'CODI_PROVINCIA', 'REFERENCIA CADASTRAL', 'ZONA CLIMATICA',
       'METRES_CADASTRE', 'ANY_CONSTRUCCIO', 'US_EDIFICI',
       'Qualificació de consum d'energia primaria no renovable',
       'Energia primària no renovable', 'Qualificacio d'emissions de CO2',
       'Emissions de CO2', 'Consum d'energia final',
       'Cost anual aproximat d'energia per habitatge', 'VEHICLE ELECTRIC',
       'SOLAR TERMICA', 'SOLAR FOTOVOLTAICA', 'SISTEMA BIOMASSA',
       'XARXA DISTRICTE', 'ENERGIA GEOTERMICA', 'INFORME_INS_TECNICA_EDIFICI',
       'Eina de certificacio', 'VALOR AILLAMENTS', 'VALOR FINESTRES',
       'Motiu de la certificacio', 'VALOR AILLAMENTS CTE',
       'VALOR FINESTRES CTE', 'UTM_X', 'UTM_Y', 'Normativa construcció',
       'Tipus Tramit', 'TIPUS_TERCIARI', 'Qualificació emissions calefacció',
       'Emissions cale

In [26]:
df_big = df_big_raw.copy()

In [27]:
for column in df_big_raw.columns:
    df_big[column.lower().join("_")] = df_big[column]


In [24]:
df_big.columns

Index(['NUM_CAS', 'ADREÇA', 'NUMERO', 'ESCALA', 'PIS', 'PORTA', 'CODI_POSTAL',
       'POBLACIO', 'COMARCA', 'NOM_PROVINCIA', 'CODI_POBLACIO', 'CODI_COMARCA',
       'CODI_PROVINCIA', 'REFERENCIA CADASTRAL', 'ZONA CLIMATICA',
       'METRES_CADASTRE', 'ANY_CONSTRUCCIO', 'US_EDIFICI',
       'Qualificació de consum d'energia primaria no renovable',
       'Energia primària no renovable', 'Qualificacio d'emissions de CO2',
       'Emissions de CO2', 'Consum d'energia final',
       'Cost anual aproximat d'energia per habitatge', 'VEHICLE ELECTRIC',
       'SOLAR TERMICA', 'SOLAR FOTOVOLTAICA', 'SISTEMA BIOMASSA',
       'XARXA DISTRICTE', 'ENERGIA GEOTERMICA', 'INFORME_INS_TECNICA_EDIFICI',
       'Eina de certificacio', 'VALOR AILLAMENTS', 'VALOR FINESTRES',
       'Motiu de la certificacio', 'VALOR AILLAMENTS CTE',
       'VALOR FINESTRES CTE', 'UTM_X', 'UTM_Y', 'Normativa construcció',
       'Tipus Tramit', 'TIPUS_TERCIARI', 'Qualificació emissions calefacció',
       'Emissions cale